In [1]:
print('chain')

chain


In [3]:
import os
import openai

from openai import OpenAI


from dotenv import load_dotenv
load_dotenv('myenv/.env')
a=os.environ.get('OPENAI_API_KEY')

In [4]:
client=OpenAI(api_key=a)

In [5]:
llm_model='gpt-3.5-turbo-0301'

In [6]:
print(llm_model)

gpt-3.5-turbo-0301


In [7]:
import pandas as pd
df=pd.read_csv('data.csv')

/var/folders/4d/gf6xkqsd5tx6m_zltry403dc0000gn/T/ipykernel_74393/8050215.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
df.head()

,Product,Review
0,Laptop,This laptop is amazing! It's fast and has a gr...
1,Smartphone,The smartphone has a fantastic camera and slee...
2,Headphones,These headphones provide excellent sound quali...
3,Tablet,The tablet has a beautiful display and long ba...
4,Camera,The camera takes stunning photos and is easy t...


In [10]:
type('eshant')

str

In [11]:
type(4)

int

In [12]:
type(df)

pandas.core.frame.DataFrame

In [13]:
df.head(2)

,Product,Review
0,Laptop,This laptop is amazing! It's fast and has a gr...
1,Smartphone,The smartphone has a fantastic camera and slee...


In [14]:
df.describe()

,Product,Review
count,46,46
unique,46,46
top,Laptop,This laptop is amazing! It's fast and has a gr...
freq,1,1


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Product  46 non-null     object
 1   Review   46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


In [16]:
df.isnull().sum()

Product    0
Review     0
dtype: int64

### `Chains`

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [18]:
llm=ChatOpenAI(temperature=0,model=llm_model)

In [19]:
prompt=ChatPromptTemplate.from_template(
    "What is the best name to describe \
     a company that makes {product}"
)

In [20]:
chain=LLMChain(llm=llm,prompt=prompt)

In [22]:
product="Cricket Bats"
chain.run(product)

'"Willow Works"'

### `SimpleSequentialChain`

In [23]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm=ChatOpenAI(temperature=0,model=llm_model)

first_prompt=ChatPromptTemplate.from_template(
    "What is the best name to describe \
     a company that makes {product}"
)
chain_one=LLMChain(llm=llm,prompt=first_prompt)

In [29]:
### prompt template 2

second_prompt=ChatPromptTemplate.from_template(
    "Write 30 words description for the following company :{company_name}"
)
chain_twp=LLMChain(llm=llm,prompt=second_prompt)

In [30]:
overall_simple_chain=SimpleSequentialChain(chains=[chain_one,chain_twp],verbose=True)

In [31]:
overall_simple_chain.run("Cricket Bats")



> Entering new SimpleSequentialChain chain...
"Willow Works"
Willow Works is a creative agency that specializes in branding, web design, and digital marketing. They help businesses build their online presence and reach their target audience through innovative strategies.

> Finished chain.


'Willow Works is a creative agency that specializes in branding, web design, and digital marketing. They help businesses build their online presence and reach their target audience through innovative strategies.'

In [32]:
overall_simple_chain.run("Headphones")



> Entering new SimpleSequentialChain chain...
"Soundwave"
Soundwave is a music discovery and sharing platform that allows users to connect with friends, discover new music, and share their favorite songs and playlists across multiple platforms.

> Finished chain.


'Soundwave is a music discovery and sharing platform that allows users to connect with friends, discover new music, and share their favorite songs and playlists across multiple platforms.'

### `Sequential Chain`

In [33]:
from langchain.chains import SequentialChain

In [35]:
first_prompt=ChatPromptTemplate.from_template(
    "Translate the following review to english: {Review}"
)

chain_one=LLMChain(llm=llm,prompt=first_prompt,output_key="English_Review")

In [36]:
second_prompt=ChatPromptTemplate.from_template(
    "Can you summarise the following review in 1 sentence : {English_Review}"
)

In [37]:
chain_two=LLMChain(llm=llm,prompt=second_prompt,output_key="summary")

In [38]:
third_prompt=ChatPromptTemplate.from_template(
    "What language is the following review {Review}"
)
chain_three=LLMChain(llm=llm,prompt=third_prompt,output_key="language")

In [40]:
fourth_prompt=ChatPromptTemplate.from_template(
    "Write a follow up response of the following"
    "summary in the specific language"
    "Summary :{summary} \n\n Language:{language}"
)

chain_four=LLMChain(llm=llm,prompt=fourth_prompt,output_key="followup_message")

In [42]:
overall_chain=SequentialChain(
    chains=[chain_one,chain_two,chain_three,chain_four],
    input_variables=['Review'],
    output_variables=['English_Review','summary','followup_message'],verbose=True
)

In [43]:
overall_chain('Dieses Produkt ist wirklich beeindruckend! Es erfüllt alle meine Erwartungen und übertrifft sie sogar. Die Qualität ist hervorragend und ich kann es jedem empfehlen, der nach einer zuverlässigen Lösung sucht')



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Dieses Produkt ist wirklich beeindruckend! Es erfüllt alle meine Erwartungen und übertrifft sie sogar. Die Qualität ist hervorragend und ich kann es jedem empfehlen, der nach einer zuverlässigen Lösung sucht',
 'English_Review': 'This product is truly impressive! It meets all of my expectations and even exceeds them. The quality is excellent and I can recommend it to anyone looking for a reliable solution.',
 'summary': 'The product is highly recommended due to its impressive quality that meets and exceeds expectations.',
 'followup_message': 'Zusammenfassung: Das Produkt wird aufgrund seiner beeindruckenden Qualität, die Erwartungen erfüllt und übertrifft, sehr empfohlen.\n\nAls KI-Sprachassistent kann ich sagen, dass das Produkt aufgrund seiner hervorragenden Qualität sehr empfehlenswert ist. Es erfüllt nicht nur die Erwartungen, sondern übertrifft sie sogar. Wenn Sie auf der Suche nach einem Produkt sind, das Ihnen ein hervorragendes Benutzererlebnis bietet, dann ist die